In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}")

✅ Gemini API key setup complete.


In [3]:
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
# Define helper functions that will be reused throughout the notebook

def show_python_code_and_result(response):
    for i in range(len(response)):
        # Check if the response contains a valid function call result from the code executor
        if (
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [6]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [9]:
!pip install -q fpdf 
print("✅ FPDF successfully installed")

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=1089c8e2c160c531e355d9ccd6ca3b07e3d47de4c6b8de82af67b74fcbc42bc5
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [15]:
enhanced_pdf_agent = LlmAgent(
    name="enhanced_pdf_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are the PDF agent. You handle two tools: 
    - generate_pdf: Create a Unicode-safe PDF from text or HTML content.
    - show_pdf: Display a PDF inline in Kaggle/Colab notebooks.

    Rules:
    1. If the user intent is to create, export, or save content as a PDF, call generate_pdf with arguments:
       - title (string)
       - html_body (string)
       - language_tag (string, optional, default "auto")
       Return the generated PDF file path.

    2. If the user intent is to view, open, or display a PDF, call show_pdf with arguments:
       - pdf_file (string filename or path)
       - width (int, default 800)
       - height (int, default 600)
       Render the PDF inline in the notebook.

    3. After generating a PDF, offer the option to immediately call show_pdf so the user can view the result without needing a separate request.
    4. Always validate inputs using schemas before calling tools. 
       - Reject or request clarification if required fields are missing.
       - Confirm before overwriting existing files.

    5. Keep responsibilities separate:
       - generate_pdf only creates and returns the file path.
       - show_pdf only displays the file inline.
       - Together, they provide a complete "create + view" workflow.
       """,
     tools=[
        generate_pdf,
        show_pdf,
        AgentTool(agent=pdf_agent),  # Using another agent as a tool!
    ],
)
print("✅ enhanced pdf agent created")

✅ enhanced pdf agent created


In [27]:
!pip install -q streamlit 
print("✅ installed successfully")

✅ installed successfully


In [40]:
%%writefile streamlit_app.py
import streamlit as st
from fpdf import FPDF
import os
import base64

# --- Tool definitions ---
def generate_pdf(title: str, html_body: str, language_tag: str = "auto") -> str:
    font_path = os.path.join(os.path.dirname(__file__), "DejaVuSans.ttf")
    pdf = FPDF()
    pdf.add_page()
    pdf.add_font("DejaVu", "", font_path, uni=True)
    pdf.set_font("DejaVu", size=12)
    pdf.multi_cell(0, 10, txt=html_body)
    filename = f"{title.replace(' ', '_')}.pdf"
    pdf.output(filename)
    return filename


def show_pdf(pdf_file: str, width: int = 800, height: int = 600):
    if not os.path.exists(pdf_file):
        st.error(f"File '{pdf_file}' not found. Please generate or upload it first.")
    else:
        # Read PDF and encode to base64
        with open(pdf_file, "rb") as f:
            base64_pdf = base64.b64encode(f.read()).decode("utf-8")

        # Embed PDF in an iframe
        pdf_display = f'<iframe src="data:application/pdf;base64,{base64_pdf}" width="{width}" height="{height}" type="application/pdf"></iframe>'
        st.markdown(pdf_display, unsafe_allow_html=True)
        
# --- Agent wrapper ---
class LlmAgent:
    def __init__(self):
        self.instructions = """
        You are the PDF agent. You handle two tools:
        - generate_pdf: Create a Unicode-safe PDF from text/HTML.
        - show_pdf: Display a PDF inline in the notebook/Streamlit UI.
        Decide which tool to call based on the user prompt.
        """

    def handle(self, prompt: str):
        prompt_lower = prompt.lower()
        if "generate pdf" in prompt_lower or "create pdf" in prompt_lower:
            # Extract simple defaults; could be extended with NLP parsing
            title = "My Report"
            body = prompt
            pdf_path = generate_pdf(title=title, html_body=body)
            return f"✅ Generated PDF: {pdf_path}", pdf_path
        elif "show pdf" in prompt_lower or "view pdf" in prompt_lower:
            pdf_file = "My_Report.pdf"
            show_pdf(pdf_file=pdf_file, width=900, height=700)
            return f"📄 Displaying PDF: {pdf_file}", None
        else:
            return "⚠️ Agent did not recognize intent. Try 'generate pdf' or 'show pdf'.", None

# --- Streamlit UI ---
st.title("📄 PDF Agent UI")

# Prompt input
user_prompt = st.text_area("Enter your prompt:", height=150)

if st.button("Run Agent"):
    agent = LlmAgent()
    message, pdf_path = agent.handle(user_prompt)
    st.write(message)
    if pdf_path and os.path.exists(pdf_path):
        st.download_button("Download PDF", open(pdf_path, "rb"), file_name=pdf_path, mime="application/pdf")

st.title("📄 PDF Agent UI")
st.write("Hello from Streamlit inside Kaggle!")


Overwriting streamlit_app.py
